In [1]:
import pandas as pd
from INEGIpy import Indicadores

## Muestra del catalogo

In [2]:
df = pd.read_excel('../catalogo/catalogoCompletoINEGI.xlsx')
df.tail()

,Variables,Claves,FechaActualizacion,Nivel1
249124,Sector externo > Importaciones y exportaciones...,229495,Fecha de última actualización: 2023-11-17,Sector externo
249125,Sector externo > Importaciones y exportaciones...,229496,Fecha de última actualización: 2023-11-17,Sector externo
249126,Sector externo > Importaciones y exportaciones...,229497,Fecha de última actualización: 2023-11-17,Sector externo
249127,Sector externo > Importaciones y exportaciones...,229498,Fecha de última actualización: 2023-11-17,Sector externo
249128,Sector externo > Importaciones y exportaciones...,229499,Fecha de última actualización: 2023-11-17,Sector externo


In [13]:
sample5 = df.sample(5, random_state=42)
sample5

,Variables,Claves,FechaActualizacion,Nivel1
62633,Manufacturas > Encuesta anual de la industria ...,692895,Fecha de última actualización: 2022-12-06,Manufacturas
30650,Manufacturas > Encuesta mensual de la industri...,648292,Fecha de última actualización: 2023-05-12,Manufacturas
71412,Manufacturas > Encuesta mensual de la industri...,703200,Fecha de última actualización: 2023-12-14,Manufacturas
64117,Manufacturas > Encuesta anual de la industria ...,694379,Fecha de última actualización: 2022-12-06,Manufacturas
22176,Manufacturas > Encuesta mensual de la industri...,639372,Fecha de última actualización: 2023-05-12,Manufacturas


In [15]:
sample5.to_excel("../catalogo/muestra5.xlsx")

In [22]:
sample5[["Variables"]].to_excel("../catalogo/muestra5-rutas.xlsx", index=False)

In [24]:
sample5[["Claves"]].to_excel("../catalogo/muestra5-claves.xlsx", index=False)

## Construir los catalogos por rutas y claves

In [ ]:
import os
import sys
import pandas as pd 
from INEGIpy import Indicadores

catalogo_path: str = "catalogo/catalogoCompletoINEGI.xlsx"
catalogo: pd.DataFrame = pd.read_excel(catalogo_path)

def construir_catalogo(formato: str):
  if formato.strip() == "Rutas":
    columna_formato = "Variables"
  elif formato.strip() == "Claves":
    columna_formato = "Claves"
  return catalogo[["Variables", "Claves"]].set_index(columna_formato).squeeze()

## Ejemplo de uso

In [40]:
def obtener_serie(ruta_archivo: str, formato:str, token:str = "f6a7b69c-5c48-bf0c-b191-5ca98c6a6cc0"):
  # Tomamos siempre la primera columna
  variables_usuario: pd.Series = pd.read_excel(ruta_archivo).iloc[:,0]
  catalogo_se: pd.Series = construir_catalogo(formato)
  
  # Se instancia la interfaz que se counicara con INEGI
  inegi = Indicadores(token)  

  if formato == "Rutas":
    #Para cada variable tendremos que sacar su clave y nombre de la variable
    claves_variables =  variables_usuario.apply(lambda x: catalogo_se[x])
    nombres_variables = variables_usuario.apply(lambda x: x.split(">")[-1])

    # Se obtiene la serie a partir de la API
    variables_df = inegi.obtener_df(indicadores=claves_variables, nombres=nombres_variables)
  
  elif formato == "Claves":
    claves_variables =  variables_usuario
    nombres_variables = variables_usuario.apply(lambda x: catalogo_se[x].split(">")[-1])
    variables_df = inegi.obtener_df(indicadores=claves_variables, nombres=nombres_variables)
  return variables_df

In [19]:
rutas_examples = pd.read_excel("catalogo/muestra5-rutas.xlsx")
rutas_examples

,Variables
0,Manufacturas > Encuesta anual de la industria ...
1,Manufacturas > Encuesta mensual de la industri...
2,Manufacturas > Encuesta mensual de la industri...
3,Manufacturas > Encuesta anual de la industria ...
4,Manufacturas > Encuesta mensual de la industri...


In [42]:
obtener_serie('catalogo/muestra5-rutas.xlsx', "Rutas")

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/INEGIpy/_indicadores.py:108: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df.set_index(pd.to_datetime(df.fechas),inplace=True, drop=True)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/INEGIpy/_indicadores.py:108: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df.set_index(pd.to_datetime(df.fechas),inplace=True, drop=True)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/INEGIpy/_indicadores.py:108: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify

Variables,311211 Beneficio del arroz (Miles de pesos),Valor total de la clase (Miles de pesos corrientes),Total 3121 Industria de las bebidas (Número de personas),"316999 Fabricación de otros productos de cuero, piel y materiales sucedáneos (Miles de pesos)",332320 Fabricación de productos de herrería (Miles de pesos corrientes)
fechas,,,,,
2013-01-01,76.0,54599.0,NaN,1196.0,231972.0
2013-02-01,NaN,40839.0,NaN,NaN,245660.0
2013-03-01,NaN,45826.0,NaN,NaN,240156.0
2013-04-01,NaN,57390.0,NaN,NaN,236798.0
2013-05-01,NaN,59118.0,NaN,NaN,255008.0
...,...,...,...,...,...
2023-06-01,NaN,NaN,151922.0,NaN,NaN
2023-07-01,NaN,NaN,153384.0,NaN,NaN
2023-08-01,NaN,NaN,154333.0,NaN,NaN


In [27]:
claves_examples = pd.read_excel("catalogo/muestra5-claves.xlsx")
claves_examples

,Claves
0,692895
1,648292
2,703200
3,694379
4,639372


In [43]:
df = obtener_serie('catalogo/muestra5-claves.xlsx', "Claves")

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/INEGIpy/_indicadores.py:108: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df.set_index(pd.to_datetime(df.fechas),inplace=True, drop=True)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/INEGIpy/_indicadores.py:108: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df.set_index(pd.to_datetime(df.fechas),inplace=True, drop=True)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/INEGIpy/_indicadores.py:108: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify

In [54]:
df

Claves,311211 Beneficio del arroz (Miles de pesos),Valor total de la clase (Miles de pesos corrientes),Total 3121 Industria de las bebidas (Número de personas),"316999 Fabricación de otros productos de cuero, piel y materiales sucedáneos (Miles de pesos)",332320 Fabricación de productos de herrería (Miles de pesos corrientes)
fechas,,,,,
2013-01-01,76.0,54599.0,NaN,1196.0,231972.0
2013-02-01,NaN,40839.0,NaN,NaN,245660.0
2013-03-01,NaN,45826.0,NaN,NaN,240156.0
2013-04-01,NaN,57390.0,NaN,NaN,236798.0
2013-05-01,NaN,59118.0,NaN,NaN,255008.0
...,...,...,...,...,...
2023-06-01,NaN,NaN,151922.0,NaN,NaN
2023-07-01,NaN,NaN,153384.0,NaN,NaN
2023-08-01,NaN,NaN,154333.0,NaN,NaN


In [56]:
df.set_index(pd.to_datetime(df.index.get_level_values("fechas")),inplace=True, drop=True)

In [57]:
df

Claves,311211 Beneficio del arroz (Miles de pesos),Valor total de la clase (Miles de pesos corrientes),Total 3121 Industria de las bebidas (Número de personas),"316999 Fabricación de otros productos de cuero, piel y materiales sucedáneos (Miles de pesos)",332320 Fabricación de productos de herrería (Miles de pesos corrientes)
fechas,,,,,
2013-01-01,76.0,54599.0,NaN,1196.0,231972.0
2013-02-01,NaN,40839.0,NaN,NaN,245660.0
2013-03-01,NaN,45826.0,NaN,NaN,240156.0
2013-04-01,NaN,57390.0,NaN,NaN,236798.0
2013-05-01,NaN,59118.0,NaN,NaN,255008.0
...,...,...,...,...,...
2023-06-01,NaN,NaN,151922.0,NaN,NaN
2023-07-01,NaN,NaN,153384.0,NaN,NaN
2023-08-01,NaN,NaN,154333.0,NaN,NaN


In [55]:
df.index.get_level_values("fechas")

DatetimeIndex(['2013-01-01', '2013-02-01', '2013-03-01', '2013-04-01',
               '2013-05-01', '2013-06-01', '2013-07-01', '2013-08-01',
               '2013-09-01', '2013-10-01',
               ...
               '2023-01-01', '2023-02-01', '2023-03-01', '2023-04-01',
               '2023-05-01', '2023-06-01', '2023-07-01', '2023-08-01',
               '2023-09-01', '2023-10-01'],
              dtype='datetime64[ns]', name='fechas', length=130, freq='MS')

## Ver indicadores

In [9]:
df.query("Variables == 'Ocupación, empleo y remuneraciones > Población ocupada, subocupada y desocupada (resultados trimestrales de la ENOE, 15 años y más) > Valores absolutos > Nacional > Mujeres > Población desocupada > Duración del desempleo > Más de 1 mes hasta 3 meses (Número de personas)'")

,Variables,Claves,FechaActualizacion,Nivel1
635,"Ocupación, empleo y remuneraciones > Población...",44706,Fecha de última actualización: 2023-11-27,"Ocupación, empleo y remuneraciones"
15158,"Ocupación, empleo y remuneraciones > Población...",447069,Fecha de última actualización: 2023-11-27,"Ocupación, empleo y remuneraciones"


In [10]:
token:str = "f6a7b69c-5c48-bf0c-b191-5ca98c6a6cc0"
inegi = Indicadores(token)

In [42]:
inegi.obtener_df(indicadores='444674')

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/INEGIpy/_indicadores.py:108: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df.set_index(pd.to_datetime(df.fechas),inplace=True, drop=True)


,444674
fechas,
2005-01-01,80.567600
2005-02-01,80.286384
2005-03-01,80.166288
2005-04-01,79.546034
2005-05-01,80.072435
...,...
2023-07-01,76.999482
2023-08-01,76.746354
2023-09-01,76.019123


## Estructura de la API

Sintaxis: 

https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/[IdIndicador]/[Idioma]/[Área Geográfica]/[Recientes]/[Fuente de datos]/[Versión][Token]?type=[Formato]

URL: 

https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/1002000001/es/00/true/BISE/2.0/[Aquí va tu Token]?type=json


In [34]:
import requests

# Banco de Indicadores
indicador = "44706"
# Definir la URL de la API
idioma = "es"
area_geografica = "00"
recientes = "true"
fuente_datos = "BIE"
version = "2.0"
token = "f6a7b69c-5c48-bf0c-b191-5ca98c6a6cc0"
formato = "json"
url = f'https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/{indicador}/{idioma}/{area_geografica}/{recientes}/{fuente_datos}/{version}/{token}?type={formato}'
url

'https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/44706/es/00/true/BIE/2.0/f6a7b69c-5c48-bf0c-b191-5ca98c6a6cc0?type=json'

In [33]:
# Banco de información Economica (BIE)

# url = f'https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/{indicador}/{idioma}/{area_geografica}/{recientes}/{fuente_datos}/{version}/{token}?type={formato}'
# #url = f"https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/1002000001/es/00/true/BISE/2.0/{token}?type=json"
# url

In [35]:
# Realizar la solicitud GET
response = requests.get(url)

# Verificar si la solicitud fue exitosa (código de estado 200)
if response.status_code == 200:
    # Acceder a los datos de la respuesta en formato JSON
    datos = response.json()
    print(datos)
else:
    print('Error en la solicitud. Código de estado:', response.status_code)


{'Header': {'Name': 'Datos compactos BIE', 'Email': 'atencion.usuarios@inegi.org.mx'}, 'Series': [{'INDICADOR': '44706', 'FREQ': '2', 'TOPIC': '11601250007000010017000400020011', 'UNIT': '26', 'UNIT_MULT': None, 'NOTE': '', 'SOURCE': '17', 'LASTUPDATE': '', 'STATUS': None, 'OBSERVATIONS': [{'TIME_PERIOD': '1985', 'OBS_VALUE': '120', 'OBS_EXCEPTION': '', 'OBS_STATUS': '', 'OBS_SOURCE': '17', 'OBS_NOTE': None, 'COBER_GEO': '00'}]}]}


In [62]:
import requests

# Banco de Indicadores
#indicador = "444674"
indicador = "733671"
# Definir la URL de la API
idioma = "es"
area_geografica = "00"
recientes = "false"
fuente_datos = "BIE"
version = "2.0"
token = "f6a7b69c-5c48-bf0c-b191-5ca98c6a6cc0"
formato = "json"
url = f'https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/{indicador}/{idioma}/{area_geografica}/{recientes}/{fuente_datos}/{version}/{token}?type={formato}'
print(url)

https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/733671/es/00/false/BIE/2.0/f6a7b69c-5c48-bf0c-b191-5ca98c6a6cc0?type=json


In [ ]:
# https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/733671/es/0700/false/BISE/2.0/c64be54e-1842-acb9-0843-baad4ab4aa56?type=json


'https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/444674/es/00/true/BIE/2.0/f6a7b69c-5c48-bf0c-b191-5ca98c6a6cc0?type=jso'
"https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/444674/es/0700/false/BIE/2.0/c64be54e-1842-acb9-0843-baad4ab4aa56?type=json"

In [52]:
# URL que te da la INEGIpy
#url = "https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/733671/es/0700/false/BIE/2.0/c64be54e-1842-acb9-0843-baad4ab4aa56?type=json"
#url = "https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/444674/es/0700/false/BIE/2.0/c64be54e-1842-acb9-0843-baad4ab4aa56?type=json"
#response = requests.get(url)
#response

In [40]:
#url = "https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/733671/es/0700/false/BISE/2.0/c64be54e-1842-acb9-0843-baad4ab4aa56?type=json"
#response = requests.get(url)
#response


In [35]:
#import json 
#json.loads(response.text)["Series"][0].pop('OBSERVATIONS')

In [63]:
# Realizar la solicitud GET
print(url)
response = requests.get(url)

# Verificar si la solicitud fue exitosa (código de estado 200)
if response.status_code == 200:
    # Acceder a los datos de la respuesta en formato JSON
    datos = response.json()
    print(datos)
else:
    print('Error en la solicitud. Código de estado:', response.status_code)


https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/733671/es/00/false/BIE/2.0/f6a7b69c-5c48-bf0c-b191-5ca98c6a6cc0?type=json
{'Header': {'Name': 'Datos compactos BIE', 'Email': 'atencion.usuarios@inegi.org.mx'}, 'Series': [{'INDICADOR': '733671', 'FREQ': '0_6', 'TOPIC': '0_1020001700100010', 'UNIT': '0_331', 'UNIT_MULT': None, 'NOTE': '0_3213', 'SOURCE': '0_17', 'LASTUPDATE': '24/11/2023 04:39:27 p. m.', 'STATUS': None, 'OBSERVATIONS': [{'TIME_PERIOD': '2023/03', 'OBS_VALUE': '25118230.778', 'OBS_EXCEPTION': '', 'OBS_STATUS': '0_1', 'OBS_SOURCE': '0_17', 'OBS_NOTE': None, 'COBER_GEO': '00'}, {'TIME_PERIOD': '2023/02', 'OBS_VALUE': '24997299.298', 'OBS_EXCEPTION': '', 'OBS_STATUS': '', 'OBS_SOURCE': '0_17', 'OBS_NOTE': None, 'COBER_GEO': '00'}, {'TIME_PERIOD': '2023/01', 'OBS_VALUE': '24251720.859', 'OBS_EXCEPTION': '', 'OBS_STATUS': '', 'OBS_SOURCE': '0_17', 'OBS_NOTE': None, 'COBER_GEO': '00'}, {'TIME_PERIOD': '2022/04', 'OBS_VALUE': '24974759.698', 'OBS_EX

In [64]:
datos = response.json()
datos

{'Header': {'Name': 'Datos compactos BIE',
  'Email': 'atencion.usuarios@inegi.org.mx'},
 'Series': [{'INDICADOR': '733671',
   'FREQ': '0_6',
   'TOPIC': '0_1020001700100010',
   'UNIT': '0_331',
   'UNIT_MULT': None,
   'NOTE': '0_3213',
   'SOURCE': '0_17',
   'LASTUPDATE': '24/11/2023 04:39:27 p. m.',
   'STATUS': None,
   'OBSERVATIONS': [{'TIME_PERIOD': '2023/03',
     'OBS_VALUE': '25118230.778',
     'OBS_EXCEPTION': '',
     'OBS_STATUS': '0_1',
     'OBS_SOURCE': '0_17',
     'OBS_NOTE': None,
     'COBER_GEO': '00'},
    {'TIME_PERIOD': '2023/02',
     'OBS_VALUE': '24997299.298',
     'OBS_EXCEPTION': '',
     'OBS_STATUS': '',
     'OBS_SOURCE': '0_17',
     'OBS_NOTE': None,
     'COBER_GEO': '00'},
    {'TIME_PERIOD': '2023/01',
     'OBS_VALUE': '24251720.859',
     'OBS_EXCEPTION': '',
     'OBS_STATUS': '',
     'OBS_SOURCE': '0_17',
     'OBS_NOTE': None,
     'COBER_GEO': '00'},
    {'TIME_PERIOD': '2022/04',
     'OBS_VALUE': '24974759.698',
     'OBS_EXCEPTION': '

In [47]:
datos

{'Header': {'Name': 'Datos compactos BIE',
  'Email': 'atencion.usuarios@inegi.org.mx'},
 'Series': [{'INDICADOR': '444674',
   'FREQ': '8',
   'TOPIC': '101000350025021000070050',
   'UNIT': '136',
   'UNIT_MULT': None,
   'NOTE': '3218',
   'SOURCE': '272',
   'LASTUPDATE': '28/12/2023 02:17:22 p. m.',
   'STATUS': None,
   'OBSERVATIONS': [{'TIME_PERIOD': '2023/11',
     'OBS_VALUE': '76.449958008632',
     'OBS_EXCEPTION': '',
     'OBS_STATUS': '',
     'OBS_SOURCE': '272',
     'OBS_NOTE': None,
     'COBER_GEO': '00'}]}]}

In [41]:
datos.pop("OBSERVATIONS")

KeyError: 'OBSERVATIONS'